In [1]:
import sys
from numpy import *
from matplotlib import pyplot as plt
import numpy as np
import copy
import csv
import pandas as pd

In [2]:
def loadDataSet(fileName):      #general function to parse tab -delimited floats
    dataMat = []                #assume last column is target value
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split(',')
        fltLine = list(map(float,curLine)) #map all elements to float()
        dataMat.append(fltLine)
    return mat(dataMat)

In [3]:
def loadCenterSet(fileName):      #general function to parse tab -delimited floats
    centerMat = []                #assume last column is target value
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split(',')
        fltLine = list(map(float,curLine)) #map all elements to float()
        centerMat.append(fltLine)
    return mat(centerMat)

In [4]:
def assignCluster(dataSet, k, centroids):
    '''For each data point, assign it to the closest centroid
    Inputs:
        dataSet: each row represents an observation and 
                 each column represents an attribute
        k:  number of clusters
        centroids: initial centroids or centroids of last iteration
    Output:
        clusterAssment: list
            assigned cluster id for each data point
    '''
    #TODO
    data=dataSet
    dataset= np.array(dataSet)
    centroids= np.array(centroids)
    clusterAssment=[]
    for i in dataset:
        initial_distance=[]
        for j in centroids:
            distance= math.dist(i,j)
            initial_distance.append(distance)
        cluster_val= initial_distance.index(min(initial_distance))
        clusterAssment.append(cluster_val)
    
    #clusterAssment_df = pd.DataFrame({'observation':data.index, 'cluster':cluster})

    return clusterAssment



In [5]:
def getCentroid(dataSet, k, clusterAssment):
    '''recalculate centroids
    Input: 
        dataSet: each row represents an observation and 
            each column represents an attribute
        k:  number of clusters
        clusterAssment: list
            assigned cluster id for each data point
    Output:
        centroids: cluster centroids
    '''
    
    #TODO
    data=pd.DataFrame(dataSet)
    data['cluster']=clusterAssment
    #clusterAssment=pd.Series(clusterAssment)
    #data.insert(loc=0, column='cluster', value=clusterAssment)
    data_sort= data.sort_values(by=['cluster'])
    centroids=[]
    for clus in range(0,k):
        new_data= data_sort.loc[data_sort['cluster'] == clus]
        for i in range(0,(len(data.columns)-1)):
            #print("new column")
            coord_list=[]
            for j in new_data.iloc[:,i]:
                coord_list.append(j)
            centroids.append(mean(coord_list))
    centroids=np.array(centroids).reshape(k,(len(data.columns)-1))

    return centroids

In [6]:
def kMeans(dataSet, T, k, centroids):
    '''
    Input:
        dataSet: each row represents an observation and 
                each column represents an attribute
        T:  number of iterations
        k:  number of clusters
        centroids: initial centroids
    Output:
        centroids: final cluster centroids
        clusterAssment: list
            assigned cluster id for each data point
    '''
    clusterAssment = [0] * len(dataSet)
    pre_clusters  = [1] * len(dataSet)

    i=1
    all_centroid=[]
    while i < T and list(pre_clusters) != list(clusterAssment):
        pre_clusters = copy.deepcopy(clusterAssment) 
        clusterAssment = assignCluster(dataSet, k, centroids )
        centroids      = getCentroid(dataSet, k, clusterAssment)
        all_centroid.append(centroids)
        i=i+1

    return centroids, clusterAssment,all_centroid

In [7]:
def saveData(save_filename, data, clusterAssment):
    clusterAssment = np.array(clusterAssment, dtype = object)[:,None]
    data_cluster = np.concatenate((data, clusterAssment), 1)
    data_cluster = data_cluster.tolist()

    with open(save_filename, 'w', newline = '') as f:
        writer = csv.writer(f)
        writer.writerows(data_cluster)
    f.close()

In [8]:
if __name__ == '__main__':
    '''if len(sys.argv) == 4:
        data_filename = sys.argv[1]
        centroid_filename = sys.argv[2]
        k = int(sys.argv[3])
    else:
        '''
        
    data_filename = 'YeastGene.csv'
    centroid_filename = 'YeastGene_Initial_Centroids.csv'
    k = 6

    save_filename = data_filename.replace('.csv', '_kmeans_cluster.csv')

    data = loadDataSet(data_filename)
    centroids = loadCenterSet(centroid_filename)
    centroids, clusterAssment,all_centroid = kMeans(data, 7, k, centroids )
    print(centroids)
    #print(all_centroid)
    saveData(save_filename, data, clusterAssment)

[[-2.41271429e-01 -1.28750000e-01  6.22500000e-02  1.73350000e-01
   2.17914286e-01  1.65169286e+00  1.90532143e+00]
 [-9.53509804e-01 -1.47164706e+00  7.75294118e-02 -1.79490196e-01
  -1.00482353e+00  1.15121569e+00  9.68803922e-01]
 [ 1.65103659e-01  9.16524390e-02 -1.03884146e-01 -5.52585366e-01
  -6.30085366e-01 -1.72318293e+00 -1.75481098e+00]
 [ 2.32857143e-02  2.50809524e-01 -2.76952381e-01 -3.64000000e-01
  -7.35428571e-01 -8.94619048e-01  7.00142857e-01]
 [-1.55555556e-03  1.56506173e-01  3.56253086e-01  7.01580247e-01
   1.00971605e+00  1.84231481e+00  1.64341975e+00]
 [-3.93289474e-02  1.53947368e-01  4.36078947e-01  1.10581579e+00
   1.44871053e+00  3.01634211e+00  2.82938158e+00]]
